# Notebook for development and testing

In [1]:
from DataLoader import *

data = ship_dataset("/home/franko/Desktop/DSelimovcProject/Dataset/", 'valid')
data[0]

(array([-0.978247,  0.47625 ,  0.277254, ...,  0.099502,  0.24331 ,
       -1.074422]), array([-0.55426 ,  0.832885,  0.348382, ...,  0.076906,  0.160946,
       -1.209159]), array([-0.05964 ,  0.023086,  0.0089  , ..., -0.002434,  0.007377,
       -0.049988])) (array(5.9759), array(8.8464), array(52.3434)) /home/franko/Desktop/DSelimovcProject/Dataset/train/IO_8180.json
